# Thermal de Broglie Wavelength

## Definition

The **thermal de Broglie wavelength** $\lambda_T$ is the characteristic quantum wavelength of a particle in thermal equilibrium at temperature $T$. It represents the de Broglie wavelength associated with a *typical thermal momentum* set by $k_B T$.

---

## Mathematical Form

For a nonrelativistic particle of mass $m$,
$$
\lambda_T
=
\frac{h}{\sqrt{2\pi m k_B T}}
=
\sqrt{\frac{2\pi \hbar^2}{m k_B T}}.
$$

---

## Origin and Interpretation

Thermal motion sets a momentum scale via equipartition,
$$
\frac{p^2}{2m} \sim \frac{1}{2}k_B T
\quad \Rightarrow \quad
p \sim \sqrt{m k_B T}.
$$

Using the de Broglie relation $\lambda = h/p$ yields a characteristic wavelength.  
The factor of $2\pi$ arises naturally when $\lambda_T$ is derived from the **canonical partition function** of a free particle,
$$
Z_1 = \frac{V}{\lambda_T^3}.
$$

Thus, $\lambda_T$ is not arbitrary—it is the normalization length scale that makes phase-space counting consistent with quantum mechanics.

---

## Physical Meaning

- Measures the **spatial extent** of a particle’s thermal wavepacket  
- Quantifies when **wavefunction overlap** becomes important  
- Sets the boundary between classical and quantum statistical behavior  

---

## Classical–Quantum Criterion

Let $n$ be the number density. Then
$$
n \lambda_T^3 \ll 1
\quad \text{(classical / Maxwell–Boltzmann regime)}
$$
$$
n \lambda_T^3 \gtrsim 1
\quad \text{(quantum degeneracy: FD or BE)}
$$

This dimensionless quantity is the standard diagnostic used in kinetic theory and statistical mechanics.

---

## Scaling Behavior

From the definition,
$$
\lambda_T \propto T^{-1/2}, \qquad
\lambda_T \propto m^{-1/2}.
$$

- Higher temperature → shorter wavelength  
- Larger mass → shorter wavelength  

Electrons therefore become quantum-degenerate at much higher temperatures than atoms.

---

## Typical Values (Order of Magnitude)

- Electron at $T = 300\ \mathrm{K}$:  
  $\lambda_T \sim 7\ \mathrm{nm}$
- Argon atom at $T = 300\ \mathrm{K}$:  
  $\lambda_T \sim 0.05\ \mathrm{nm}$

---

## Where It Appears

- Sackur–Tetrode equation (ideal-gas entropy)
- Quantum corrections to classical gases
- Bose–Einstein condensation
- Electron degeneracy in metals and plasmas
- Transport theory and evaporation kinetics

---

## Summary

The thermal de Broglie wavelength is the **quantum length scale set by temperature**.  
It controls when classical particle descriptions fail and quantum statistics become unavoidable.


In [7]:
from __future__ import annotations

from dataclasses import dataclass
import math

from physkit.constants import ConstantsSI


@dataclass(frozen=True, slots=True)
class ThermalDeBroglieWavelength:
  r"""
  Thermal de Broglie wavelength for a nonrelativistic particle.

  Definition:
      λ_T = h / sqrt(2π m k_B T) = sqrt(2π ħ^2 / (m k_B T))

  Units:
      m: kg
      T: K
      λ_T: m

  Notes:
  - This is the canonical normalization length appearing in the ideal-gas
    single-particle partition function: Z_1 = V / λ_T^3.
  - Degeneracy criterion for a gas of number density n:
        n λ_T^3 << 1  (classical / MB)
        n λ_T^3 ≳ 1   (quantum: FD/BE)
  """

  k_B = ConstantsSI.k_B
  h = ConstantsSI.h

  # ---------------------------------------------------------------------
  # Core computations
  # ---------------------------------------------------------------------
  @classmethod
  def lambda_T(cls, m: float, T: float) -> float:
    r"""
    Compute the thermal de Broglie wavelength λ_T.

    Parameters
    ----------
    m : float
        Particle mass in kg.
    T : float
        Temperature in K. Must be > 0.

    Returns
    -------
    float
        Thermal de Broglie wavelength in meters.
    """
    cls._validate(m=m, T=T)
    return cls.h / math.sqrt(2.0 * math.pi * m * cls.k_B * T)

  @classmethod
  def degeneracy_parameter(cls, n: float, m: float, T: float) -> float:
    r"""
    Compute the degeneracy parameter: n λ_T^3.

    Parameters
    ----------
    n : float
        Number density in 1/m^3. Must be >= 0.
    m : float
        Particle mass in kg.
    T : float
        Temperature in K. Must be > 0.

    Returns
    -------
    float
        Dimensionless degeneracy parameter.
    """
    if n < 0.0:
      raise ValueError(f"n must be >= 0, got n={n}")
    lam = cls.lambda_T(m=m, T=T)
    return n * lam**3

  @classmethod
  def is_classical(cls, n: float, m: float, T: float, tol: float = 1e-2) -> bool:
      r"""
      Decide whether Maxwell–Boltzmann statistics are safe.

      Uses a heuristic threshold on n λ_T^3:
          n λ_T^3 < tol  => classical (MB) regime

      Parameters
      ----------
      n : float
          Number density in 1/m^3.
      m : float
          Particle mass in kg.
      T : float
          Temperature in K.
      tol : float
          Threshold for classicality (default 1e-2).

      Returns
      -------
      bool
          True if classical regime under this tolerance.
      """
      if tol <= 0.0:
          raise ValueError(f"tol must be > 0, got tol={tol}")
      return cls.degeneracy_parameter(n=n, m=m, T=T) < tol

  # ---------------------------------------------------------------------
  # Validation helpers
  # ---------------------------------------------------------------------
  @staticmethod
  def _validate(*, m: float, T: float) -> None:
      if m <= 0.0:
          raise ValueError(f"m must be > 0 (kg), got m={m}")
      if T <= 0.0:
          raise ValueError(f"T must be > 0 (K), got T={T}")




In [8]:
# -------------------------------------------------------------------------
# Example usage
# -------------------------------------------------------------------------

# Electron at 300 K
m_e = ConstantsSI.me0 # kg
T = 300.0 # K
lam = ThermalDeBroglieWavelength.lambda_T(m=m_e, T=T)
print("λ_T (electron, 300 K) =", lam, "m")

# Degeneracy check example: ideal gas near STP number density ~ 2.5e25 m^-3
n = 2.5e25
print("n λ_T^3 =", ThermalDeBroglieWavelength.degeneracy_parameter(n=n, m=m_e, T=T))
print("classical?", ThermalDeBroglieWavelength.is_classical(n=n, m=m_e, T=T))


λ_T (electron, 300 K) = 4.303475439595208e-09 m
n λ_T^3 = 1.9924984622825448
classical? False
